<a href="https://colab.research.google.com/github/safikhanSoofiyani/CS6910-Assignment1/blob/main/CS6910_Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing necessary libraries.

In [2]:
import keras
from keras.datasets import fashion_mnist
import numpy as np
import matplotlib.pyplot as plt
import copy 


Installing and importing wandb

In [ ]:
!pip install wandb -qqq
import wandb

     |████████████████████████████████| 1.7 MB 5.2 MB/s 
     |████████████████████████████████| 180 kB 64.9 MB/s 
     |████████████████████████████████| 143 kB 68.5 MB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 


In [ ]:
#MISSING******* Plotting images via wandb************

Preparing dataset

In [8]:
from sklearn.model_selection import train_test_split

def prepdata():
  # Loading data
  (train_x,train_y),(test_x,test_y)=fashion_mnist.load_data()

  # Defining labels for data
  num_classes = 10
  labels=['T-shirt/top','Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle boot']

  print("Number of data points in train data (initially) - ", len(train_x))
  print("Number of data points in test data (initially) - ", len(test_x))


  #performing the train-validation split
  train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.1, random_state=40)
  

  print("Shape of each image - 28x28" )
  image_shape=train_x.shape[1]*train_x.shape[2]
  print("shape of each image (1D) - ",image_shape)
  
  train_image_count=len(train_x)
  val_image_count = len(val_x)
  test_image_count=len(test_x)
  
  # Creating a matrix of image data 
  # each image is represented as a row by flattening the matrix: converting (60000,28,28) tensor to (60000,784) matrix
  X_train=np.zeros((train_image_count,image_shape))
  X_val=np.zeros((val_image_count,image_shape))
  X_test=np.zeros((test_image_count,image_shape))
  
  # converting the images into grayscale by normalizing
  for i in range(train_image_count):
    X_train[i]=(copy.deepcopy(train_x[i].flatten()))/255.0 
  for i in range(val_image_count):
    X_val[i]=(copy.deepcopy(val_x[i].flatten()))/255.0
  for i in range(test_image_count):
    X_test[i]=(copy.deepcopy(test_x[i].flatten()))/255.0
  


  #Once hot encoding the label vectors
  y_train = np.zeros((train_y.size, 10))
  y_train[np.arange(train_y.size), train_y] = 1

  y_val = np.zeros((val_y.size, 10))
  y_val[np.arange(val_y.size), val_y] = 1

  y_test = np.zeros((test_y.size, 10))
  y_test[np.arange(test_y.size), test_y] = 1

  

  
  return X_train,X_val,X_test,y_train,y_val,y_test
  

Implementing Feed Forward Neural Network

In [ ]:
#Separate the functions (to be done later)

In [4]:
def nn_init(layer_sizes,init_type):
  # Layer Sizes denotes the number of neurons per layer
  # 784 is for the input layer. 
  # 32 is for the hidden layers. 
  # 10 is for the output layers

  # initializing parameters for the neural network, 
  params={}
  if(init_type=="xavier"):
    for i in range(1,len(layer_sizes)):
      norm_xav=np.sqrt(6)/np.sqrt(layer_sizes[i]+layer_sizes[i-1])
      params["w"+str(i)]=np.random.randn(layer_sizes[i],layer_sizes[i-1])*norm_xav
      params["b"+str(i)]=np.zeros((layer_sizes[i],1))

  elif(init_type=="random"):
    for i in range(1,len(layer_sizes)):
      params["w"+str(i)]=np.random.randn(layer_sizes[i],layer_sizes[i-1])
      params["b"+str(i)]=np.random.randn((layer_sizes[i],1))

  else:
    print("Enter a valid weight initilization type")

  return params


In [6]:
# Activation Functions

def sigmoid(pre_act):
  return (1/(1+np.exp(-pre_act)))

def tanh(pre_act):
  return (np.tanh(pre_act))

def relu(pre_act):
  return (np.maximum(0,pre_act))

def softmax(x):
  return(np.exp(x)/np.sum(np.exp(x)))

In [41]:
def forward_prop(X,y,params,active,layer_sizes):
  
  # Extracting only the image data not the label for the image data
  out=copy.deepcopy(X)
  out=out.reshape(-1,1)
  
  h=[out] # To save the activations for each neuron in a layer
  a=[out] # To save the preactivation for each neuron in a layer

  if(active=="sigmoid"):
    for i in range(1,len(layer_sizes)-1):
      weights=params["w"+str(i)]
      biases=params["b"+str(i)]
      
      out=np.dot(weights,out)+biases
      a.append(out)
      post_a=sigmoid(out)
      h.append(post_a)
  
  elif(active=="tanh"):
    for i in range(1,len(layer_sizes)-1):
      weights=params["w"+str(i)]
      biases=params["b"+str(i)]
      
      out=np.dot(weights,out)+biases
      a.append(out)
      post_a=tanh(out)
      h.append(post_a)
  
  elif(active=="relu"):
    for i in range(1,len(layer_sizes)-1):
      weights=params["w"+str(i)]
      biases=params["b"+str(i)]
      
      out=np.dot(weights,out)+biases
      a.append(out)
      post_a=relu(out)
      h.append(post_a)       
  else:
    print("Enter a valid activation function") 

  # Final step for forward propagation, using softmax.
  weights=params["w"+str(len(layer_sizes)-1)]
  biases=params["b"+str(len(layer_sizes)-1)]
  
  out=np.dot(weights,post_a)+biases
  a.append(out)
  y_hat=softmax(out)
  h.append(y_hat)
  
  
  #in h we  are storing values for layers right from input till output
  #h0 is input
  #in a we are storing values for layers right from input till output
  #a0 is input

  return h,a,y_hat

In [42]:
X_train,X_val,X_test,y_train,y_val,y_test = prepdata()
parameters=nn_init(layer_sizes=[3,3,3,2],init_type="xavier")
h,a,y_hat = forward_prop(np.array([2,3,5]), [0,1],parameters, "sigmoid", [3,3,3,2])



Number of data points in train data (initially) -  60000
Number of data points in test data (initially) -  10000
Shape of each image - 28x28
shape of each image (1D) -  784


In [43]:
print(y_train[1])

[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


In [44]:
print(a)

[array([[2],
       [3],
       [5]]), array([[-2.85190181],
       [ 6.18249135],
       [ 6.07477903]]), array([[-10.70717507],
       [  0.63775227],
       [-12.47237723]]), array([[ 0.55525001],
       [-0.5318536 ]])]


In [35]:
# Calculating loss 
def loss_calc(name,y_t,y_hat):
  error=0
  if(name=="sse"):
    error=np.sum(((y_t-y_hat)**2))
  elif(name=="cross_entropy"):
    error=-1*np.sum(np.multiply(y_t,np.log(y_hat)))

  return error


In [34]:
# Calculating derivatives of the acitvation functions
def derivative(A, activation):
  if activation == "sigmoid":
    return sigmoid_der(A)
  elif activation == "tanh":
    return tanh_der(A)
  elif activation == "relu":
    return relu_der(A)


def sigmoid_der(x):
  return sigmoid(x)*(1-sigmoid(x))

def tanh_der(x):
  return 1-tanh(x)**2

def relu_der(x):
  return 1. * (x>0)


In [45]:
def back_prop(y, y_hat, h, a, params, loss_type, layer_sizes, activation):
  
  #here both y_hat and y are assumed to be column vectors



  grad = {}

  if loss_type == "squared_error":
    #waiting
    k = 0

  elif loss_type == 'cross_entropy':
    #Here actually it should be one hot vector. But y does the same job
    grad["da"+str(len(layer_sizes)-1)] = -(y-y_hat)
    grad["dh"+str(len(layer_sizes)-1)] = -(y/y_hat)

  for i in range(len(layer_sizes)-1, 0, -1 ):
    print(i)
    grad["dw"+str(i)] = np.dot(grad["da"+str(i)], np.transpose(h[i-1]))
    grad["db"+str(i)] = grad["da"+str(i)]

    if i > 1:
      grad["dh"+str(i-1)] = np.dot(np.transpose(params["w"+str(i)]), grad["da"+str(i)])
      grad["da"+str(i-1)] = np.multiply(grad["dh" + str(i-1)], derivative(a[i-1],activation))
 
  return grad



In [57]:
k = np.reshape(y_train[1], (-1,1))
print(k)
print(y_train[1])

[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]]
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


In [ ]:
def vanilla_gd(X_train,y_train, ):
  parameters = nn_init([784,32,32,10], 'random')
  eta = 0.1
  max_epochs = 10000
  for t in range(1000):

    for i in range(len(X_train)):
      y = np.reshape(y_train[i], (-1,1))
      h,a,y_hat = forward_prop(X_train[i], y, parameters, "sigmoid", [784,32,32,10])
      new_grads = back_prop(y,y_hat, h,a, parameters, "cross_entropy", [784,32,32,10], "sigmoid")
      if i == 0:
        grads = copy.deepcopy(new_grads)
      #adding the gradients
      for j in range(3,0,-1):
        grads["dw"+str(i)] += new_grads["dw"+str(i)]
        grads["db"+str(i)] += new_grads["db"+str(i)]
        grads["da"+str(i)] += new_grads["da"+str(i)]
        grads["dh"+str(i)] += new_grads["dh"+str(i)]



    for j in range(3,0,-1):
      parameters["w"+str(j)] = parameters["w"+str(j)] - (eta * grads["dw"+str(j)])
      parameters["b"+str(j)] = parameters["b"+str(j)] - (eta * grads["db"+str(j)])
